In [ ]:
import keras
from keras import layers
from keras import regularizers
import numpy as np
import json
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ingredient_map = json.load(open("/content/drive/MyDrive/PizzaGenerator/mapping.json"))
mapping = list(json.load(open("/content/drive/MyDrive/PizzaGenerator/mapping.json")).keys())
vectors = json.load(open("/content/drive/MyDrive/PizzaGenerator/vectors.json"))

Mounted at /content/drive


In [332]:
pizzas = []
preferences = []

for pizza in vectors:
    for _ in range(1000):
        pizzas.append(pizza)
        preference = []
        for ingredient in pizza:
            if ingredient == 0:
                preference.append(random.choices([0,1],cum_weights=[0.95,1])[0])
            else:
                preference.append(0)
        preferences.append(preference)

In [333]:
# Input Dataset
X1 = np.matrix(pizzas)
X2 = np.matrix(preferences)

#Number of ingredients
(N,M) = X1.shape

# This is the size of our encoded representations
encoding_dim = 10

In [334]:
# Input layer
input_recipe = keras.Input(shape=(M,))
input_requirements = keras.Input(shape=(M,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='sigmoid', activity_regularizer=regularizers.l1(10e-5))(input_recipe)

#dropout = layers.Dropout(.2)(encoded)

encoded_requirements = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_requirements)

# "concat" it concatenates the requirements and the encoded recipe
concat = layers.Concatenate()([encoded, encoded_requirements])

# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(M, activation='sigmoid')(concat)

# ThresholdLayer self learn the threshold
#threshold = ThresholdLayer()(decoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(inputs=[input_recipe,input_requirements], outputs=decoded)

In [335]:
###### DECODER MODEL

# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]

encoded_input = keras.Input(shape=(encoding_dim,))

concated_input = layers.Concatenate()([encoded_input, encoded_requirements])
# Create the decoder model
decoder = keras.Model([encoded_input, input_requirements], decoder_layer(concated_input))

In [336]:
autoencoder.compile(optimizer='adam', loss="mse")

In [337]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=1)

autoencoder.fit(x=[X1, X2], y=X1, shuffle=True, epochs=20, batch_size=4000, callbacks=[callback])

Epoch 1/20
58/58 [==============================] - 2s 23ms/step - loss: 0.1952
Epoch 2/20
58/58 [==============================] - 1s 24ms/step - loss: 0.0748
Epoch 3/20
58/58 [==============================] - 1s 24ms/step - loss: 0.0432
Epoch 4/20
58/58 [==============================] - 1s 24ms/step - loss: 0.0387
Epoch 5/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0373
Epoch 6/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0367
Epoch 7/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0363
Epoch 8/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0360
Epoch 9/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0358
Epoch 10/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0355
Epoch 11/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0352
Epoch 12/20
58/58 [==============================] - 1s 24ms/step - loss: 0.0349
Epoch 13/20
58/58 [==================

In [366]:
vec = np.random.rand(1,encoding_dim);
#requirements = np.matrix(random.choices([-1,0,1],cum_weights=[0.025,0.975,1], k=M));
requirements = np.zeros((1,M))
requirements[0,0] = 1
requirements[0,3] = 1

prob = decoder.predict(x=[vec, requirements])
pizza = list(range(M))
pizza.sort(key=lambda i: prob.item(0,i), reverse = True)

hated = []
piza = []
for i in range(M):
    if requirements.item((0,i)) == 1:
        hated.append(mapping[i])
    if prob.item(0,i) > 0.8 or len(piza) < 5:
        piza.append(mapping[pizza[i]])

piza.sort()

print("hated = ", hated)
print("pizza = ", piza)

hated =  ['bbq topping', 'pineapple']
pizza =  ['black_pepper', 'mozzarella', 'olive_oil', 'salt', 'tomato']


In [368]:
# serialize model to JSON
model_json = decoder.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
decoder.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
